<a href="https://colab.research.google.com/github/Daarlens/ProBook/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_4_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧪 **Лабораторный практикум № 4**  
# **Сравнительный анализ алгоритмов кластеризации текстовых данных на русскоязычных корпусах**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Обработка естественного языка  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Получить комплексное практическое представление о **всех основных классических методах кластеризации текстов**, основанных на статистических и статических векторных представлениях, обученных на собственном корпусе. Научиться выбирать оптимальный алгоритм в зависимости от структуры данных, вычислительных ограничений и требований к интерпретируемости.

### **Задачи:**  
1. Использовать корпус из **Лабораторной работы №1** (≥10 000 документов).  
2. Применить **методы токенизации**, включая **собственную BPE-модель**, обученную в ЛР №1.  
3. Использовать **модели векторизации**, обученные в **Лабораторной работе №2**:  
   - Word2Vec (CBOW / Skip-Gram)  
   - FastText (CBOW / Skip-Gram)  
   - GloVe  
4. Реализовать и экспериментально сравнить **все основные классические методы кластеризации**:  
   - **Центроидные**: k-Means, Mini-Batch k-Means, Spherical k-Means  
   - **Плотностные**: DBSCAN, HDBSCAN  
   - **Иерархические**: агломеративная кластеризация (single, complete, average, ward)  
   - **Вероятностные**: Gaussian Mixture Models (GMM), Latent Dirichlet Allocation (LDA)  
   - **Графовые**: спектральная кластеризация  
5. Оценить качество с помощью **внутренних и внешних метрик**.  
6. Провести **визуализацию и интерпретацию** кластеров.  
7. Исследовать **устойчивость** к шуму, дисбалансу и объёму данных.  
8. Разработать **веб-интерфейс** для анализа результатов.

---

## 📚 2. Теоретические предпосылки

Кластеризация текстов — задача **обучения без учителя**, направленная на группировку документов по семантическому сходству **без разметки**. В данной работе рассматриваются **только классические, проверенные и интерпретируемые методы**, реализованные в зрелых библиотеках (`scikit-learn`, `Gensim`, `hdbscan`).

### **2.1. Токенизация (из ЛР №1)**
- Whitespace  
- Регулярные выражения  
- **Byte Pair Encoding (BPE)** — обученная на корпусе ЛР №1 модель (через `subword-nmt` или аналог)

### **2.2. Векторизация (из ЛР №2)**
- **TF-IDF**, **BM25** — статистические веса  
- **Word2Vec**, **GloVe**, **FastText** — статические эмбеддинги, обученные в ЛР №2 на том же корпусе  
> Вектор документа = среднее по токенам. Все модели — **собственные**, не предобученные извне.

### **2.3. Классические алгоритмы кластеризации**

Все методы реализованы в следующих библиотеках:

#### **Центроидные** (`scikit-learn`):
- `KMeans` — минимизирует сумму квадратов расстояний до центроидов  
- `MiniBatchKMeans` — масштабируемая версия для больших данных  
- `SphericalKMeans` — вариант для нормализованных (косинусных) пространств

#### **Плотностные** (`scikit-learn`, `hdbscan`):
- `DBSCAN` — обнаруживает кластеры произвольной формы, устойчив к выбросам  
- `HDBSCAN` — улучшенная иерархическая версия DBSCAN, **автоматически определяет число кластеров**

#### **Иерархические** (`scikit-learn`):
- `AgglomerativeClustering` с:
  - `linkage='ward'` (требует евклидовой метрики)  
  - `linkage='average'`, `'complete'`, `'single'` (работают с `metric='cosine'`)

#### **Вероятностные**:
- `GaussianMixture` (`scikit-learn`) — мягкое назначение точек кластерам  
- `LdaModel` (`Gensim`) — генеративная тематическая модель, рассматриваемая как кластеризатор по темам

#### **Графовые** (`scikit-learn`):
- `SpectralClustering` — кластеризация на основе собственных векторов матрицы схожести

> ⚠️ Все алгоритмы работают с `numpy.ndarray`, что делает их **универсальными** для любого векторного представления.

---

## 🧪 3. Методика эксперимента

### **3.1. Подготовка данных**
- Используйте корпус из ЛР №1.  
- Реализуйте `text_preprocessing.py` с поддержкой:
  - Очистки, лемматизации (`spaCy` или `pymorphy2`),  
  - Токенизации: whitespace, regex, **BPE (из ЛР №1)**.

> 💡 **spaCy** поддерживает 64-битные Python 3.7+ на Linux, macOS, Windows. Для лемматизации русского языка может потребоваться внешний морфологический анализатор.

### **3.2. Векторизация**
- Реализуйте `text_to_vector.py` с:
  - `TfidfVectorizer` (scikit-learn)  
  - `BM25Okapi` (rank-bm25)  
  - Загрузкой моделей из ЛР №2: `Word2Vec`, `FastText`, `GloVe` (через `Gensim`)  
- Нормализуйте векторы (`L2`) при использовании cosine similarity.

> 💡 **Gensim** — сверхбыстрая библиотека для эмбеддингов (C-ядра, параллелизм, streaming). Поддерживает Python 3.8+ и работает на всех платформах.

### **3.3. Кластеризация**
Реализуйте `clustering.py`, включающий **все перечисленные выше методы**.  
Для каждого:
- Подберите гиперпараметры (`k`, `eps`, `min_samples`, `n_components` и др.)  
- Примените стратифицированную или репрезентативную выборку при оценке

### **3.4. Оценка качества**
- **Внутренние метрики**: Silhouette, Calinski-Harabasz, Davies-Bouldin  
- **Внешние метрики** (если есть тематическая разметка из ЛР №3): ARI, NMI, V-measure  
- Постройте графики зависимости метрик от параметров

### **3.5. Интерпретация**
- Для TF-IDF: топ-10 слов на кластер  
- Для эмбеддингов: ближайшие слова к центроиду (через `model.most_similar()`)  
- Визуализация: UMAP → 2D scatter plot с цветами кластеров

> 📌 **LIME и SHAP** могут применяться **только если кластеризация используется как признак в downstream-задаче** (например, кластер → признак для классификатора). В чистой кластеризации они **не применимы напрямую**, но могут использоваться для анализа косвенных эффектов.

### **3.6. AutoML для кластеризации?**
> ❗ **Важно**: **AutoML-фреймворки (Auto-sklearn, TPOT, H2O)** **не поддерживают задачи без учителя** (включая кластеризацию) в стандартной конфигурации.  
> Auto-sklearn поддерживает **только задачи с разметкой** (классификация, регрессия).  
> Следовательно, **AutoML в данной работе НЕ ИСПОЛЬЗУЕТСЯ**.

### **3.7. Веб-интерфейс**
- Используйте `Streamlit` или `Gradio` для:
  - выбора метода токенизации/векторизации/алгоритма,  
  - визуализации кластеров,  
  - просмотра содержимого кластеров.

---

## 🔍 4. Дополнительные задания

1. Сравнение влияния BPE (из ЛР №1) и whitespace на качество  
2. Использование LDA как признакового пространства для k-Means  
3. Граф кластеров: построение матрицы схожести → community detection  
4. Автоматическое определение числа кластеров (silhouette analysis, gap statistic)

---

## 📄 5. Требования к отчёту

- Укажите, что BPE и эмбеддинги — **собственные, обученные в ЛР №1 и №2**  
- Приведите **таблицу сравнения всех методов** по качеству и времени  
- Включите **визуализации** и **примеры документов** из кластеров  
- Ссылки на GitHub и веб-демо  
- Рефлексия:  
  - Какой метод кластеризации показал наилучшие результаты для русского языка?  
  - Как влияет выбор векторизации на форму кластеров?  
  - Почему AutoML не применим к кластеризации?

---

## 📖 6. Используемые библиотеки

| Библиотека | Назначение | Примечание |
|-----------|-----------|-----------|
| `Gensim` | Word2Vec, FastText, GloVe, LDA | Супербыстрая, streaming, кроссплатформенная |
| `scikit-learn` | TF-IDF, k-Means, DBSCAN, GMM, спектральная кластеризация | Стандарт де-факто |
| `hdbscan` | HDBSCAN | Для кластеров произвольной формы |
| `spaCy` | Лемматизация, базовая токенизация | Требует установки языковых моделей |
| `subword-nmt` | BPE | Для обучения и применения BPE |
| `umap-learn` | Визуализация | Альтернатива t-SNE |
| `Streamlit` / `Gradio` | Веб-интерфейс | Быстрая разработка |



## 🧪 3. Методика эксперимента

### **3.1. Подготовка данных**
- Используйте корпус из ЛР №1.  
- Реализуйте `text_preprocessing.py` с поддержкой:
  - Очистки, лемматизации (`spaCy` или `pymorphy2`),  
  - Токенизации: whitespace, regex, **BPE (из ЛР №1)**.

> 💡 **spaCy** поддерживает 64-битные Python 3.7+ на Linux, macOS, Windows. Для лемматизации русского языка может потребоваться внешний морфологический анализатор.


In [ ]:
!pip install --upgrade --quiet gensim


In [ ]:
!pip install pymorphy3


In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import pymorphy3
import sentencepiece as spm
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec, FastText


In [ ]:
# 1) Установка корус
!pip install corus

# 2) Скачивание корпуса Lenta
!wget -q https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

# 3) Чтение и подсчёт записей
import gzip
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = list(load_lenta(path))
print("Всего записей в корпусе Lenta:", len(records))

# 4) Посмотреть первые 5 текстов (например, заголовки и тексты)
for rec in records[:5]:
    print("URL:", rec.url)
    print("Заголовок:", rec.title)
    print("Текст:", rec.text[:200], "…")
    print("-" * 50)




In [ ]:
import gensim
from tqdm import tqdm
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def preprocess(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Токенизация (простой способ через gensim)
    tokens = gensim.utils.simple_preprocess(text, min_len=2, max_len=15)

    # 3. Лемматизация
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]

    # 4. Возврат строки
    return " ".join(lemmas)

# Применяем ко всем текстам корпуса (можно выбрать только текст, без заголовков)
texts = [rec.text for rec in records[:15000]]  # первые 15 000 документов для ЛР
preprocessed_texts = [preprocess(t) for t in tqdm(texts)]


In [ ]:
# Посмотрим первые 3 документа после предобработки
for i in range(3):
    print(preprocessed_texts[i])
    print("-" * 50)


In [ ]:
import sentencepiece as spm

# 1️⃣ Сохраняем предобработанные тексты во временный файл
with open("corpus_preprocessed.txt", "w", encoding="utf-8") as f:
    for line in preprocessed_texts:
        f.write(line + "\n")

# 2️⃣ Обучаем BPE модель
spm.SentencePieceTrainer.Train(
    input='corpus_preprocessed.txt',
    model_prefix='bpe',
    vocab_size=8000,    # размер словаря, можно увеличить
    model_type='bpe',   # тип модели
    character_coverage=1.0  # полностью покрыть все символы
)

# 3️⃣ Загружаем обученную BPE модель
sp = spm.SentencePieceProcessor()
sp.load('bpe.model')

# 4️⃣ Пример токенизации BPE
example = "Вице-премьер рассказал о смертности от рака в регионах"
print(sp.encode(example, out_type=str))


In [ ]:
from gensim.models import Word2Vec, FastText

# Список токенов для каждого документа
sentences = [doc.split() for doc in preprocessed_texts]

# 1️⃣ Word2Vec
w2v = Word2Vec(sentences, vector_size=100, window=5, sg=1, min_count=2, workers=4)
print("Пример похожих слов к 'рак':", w2v.wv.most_similar("рак", topn=5))

# 2️⃣ FastText
ft = FastText(sentences, vector_size=100, window=5, sg=1, min_count=2, workers=4)
print("Пример похожих слов к 'рак':", ft.wv.most_similar("рак", topn=5))

# 3️⃣ Вектор документа = среднее по токенам
import numpy as np

def doc_vector(model, doc):
    vecs = [model.wv[word] for word in doc.split() if word in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

X_w2v = np.array([doc_vector(w2v, doc) for doc in preprocessed_texts])
X_ft = np.array([doc_vector(ft, doc) for doc in preprocessed_texts])

print("Word2Vec матрица:", X_w2v.shape)
print("FastText матрица:", X_ft.shape)


In [ ]:
%%writefile text_preprocessing.py
import gensim
from pymorphy3 import MorphAnalyzer
import sentencepiece as spm

morph = MorphAnalyzer()
sp = spm.SentencePieceProcessor()
sp.load("bpe.model")  # если есть готовая BPE

def preprocess(text):
    tokens = gensim.utils.simple_preprocess(text)
    lemmas = [morph.parse(t)[0].normal_form for t in tokens]
    return " ".join(lemmas)

def tokenize_bpe(text):
    return sp.encode(text, out_type=str)



In [ ]:
from text_preprocessing import preprocess, tokenize_bpe

doc = "Вице-премьер рассказал о смертности"
tokens = preprocess(doc)
bpe_tokens = tokenize_bpe(doc)

print("Preprocessed:", tokens)
print("BPE tokens:", bpe_tokens)


In [ ]:
from text_preprocessing import preprocess, tokenize_bpe
from tqdm import tqdm

# Берём текстовые части документов
texts = [rec.text for rec in records[:15000]]

# 1️⃣ Предобработка (очистка + лемматизация)
preprocessed_texts = [preprocess(doc) for doc in tqdm(texts)]

# 2️⃣ BPE-токенизация всего корпуса
bpe_texts = [tokenize_bpe(doc) for doc in tqdm(preprocessed_texts)]

# 3️⃣ Проверка первых документов
for i in range(3):
    print("Документ:", preprocessed_texts[i])
    print("BPE токены:", bpe_texts[i][:20])
    print("-" * 50)




### **3.2. Векторизация**
- Реализуйте `text_to_vector.py` с:
  - `TfidfVectorizer` (scikit-learn)  
  - `BM25Okapi` (rank-bm25)  
  - Загрузкой моделей из ЛР №2: `Word2Vec`, `FastText`, `GloVe` (через `Gensim`)  
- Нормализуйте векторы (`L2`) при использовании cosine similarity.

> 💡 **Gensim** — сверхбыстрая библиотека для эмбеддингов (C-ядра, параллелизм, streaming). Поддерживает Python 3.8+ и работает на всех платформах.

In [ ]:
%%writefile text_to_vector.py
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from gensim.models import Word2Vec, FastText
from sklearn.preprocessing import normalize
import gensim.downloader as api

# --------------------------
# TF-IDF
# --------------------------
def tfidf_vectorize(docs, max_features=10000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(docs)
    X = normalize(X, norm='l2')  # L2-нормализация
    return X, vectorizer

# --------------------------
# BM25
# --------------------------
def bm25_vectorize(tokenized_docs):
    bm25 = BM25Okapi(tokenized_docs)
    return bm25

# --------------------------
# Word2Vec / FastText
# --------------------------
def train_word2vec(sentences, vector_size=100, window=5, sg=1, min_count=2):
    model = Word2Vec(sentences, vector_size=vector_size, window=window,
                     sg=sg, min_count=min_count, workers=4)
    return model

def train_fasttext(sentences, vector_size=100, window=5, sg=1, min_count=2):
    model = FastText(sentences, vector_size=vector_size, window=window,
                     sg=sg, min_count=min_count, workers=4)
    return model

def doc_vector(model, doc_tokens):
    vecs = [model.wv[token] for token in doc_tokens if token in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# --------------------------
# GloVe
# --------------------------
def glove_vectorize(tokenized_docs, model_name="glove-wiki-gigaword-100"):
    glove_model = api.load(model_name)
    vectors = []
    for doc in tokenized_docs:
        vecs = [glove_model[word] for word in doc if word in glove_model]
        vectors.append(np.mean(vecs, axis=0) if vecs else np.zeros(glove_model.vector_size))
    vectors = normalize(np.array(vectors), norm='l2')
    return vectors, glove_model


In [ ]:
!pip install rank_bm25


In [ ]:
from text_to_vector import tfidf_vectorize, bm25_vectorize, train_word2vec, train_fasttext, doc_vector, glove_vectorize
from tqdm import tqdm
from rank_bm25 import BM25Okapi


# 1️⃣ TF-IDF
X_tfidf, tfidf_vectorizer = tfidf_vectorize(preprocessed_texts)

# 2️⃣ BM25
tokenized_docs = [doc.split() for doc in preprocessed_texts]
bm25 = bm25_vectorize(tokenized_docs)

# 3️⃣ Word2Vec
w2v_model = train_word2vec(tokenized_docs)
X_w2v = np.array([doc_vector(w2v_model, doc) for doc in tqdm(tokenized_docs)])

# 4️⃣ FastText
ft_model = train_fasttext(tokenized_docs)
X_ft = np.array([doc_vector(ft_model, doc) for doc in tqdm(tokenized_docs)])

# 5️⃣ GloVe
X_glove, glove_model = glove_vectorize(tokenized_docs)
print("Все матрицы созданы!")


In [ ]:
print("TF-IDF:", X_tfidf.shape)
print("Word2Vec:", X_w2v.shape)
print("FastText:", X_ft.shape)
print("GloVe:", X_glove.shape)


### **3.3. Кластеризация**
Реализуйте `clustering.py`, включающий **все перечисленные выше методы**.  
Для каждого:
- Подберите гиперпараметры (`k`, `eps`, `min_samples`, `n_components` и др.)  
- Примените стратифицированную или репрезентативную выборку при оценке

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan
import umap
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------
# 1️⃣ Выбор подмножества для теста
# -----------------------------
sample_size = 2000
X_sample = X_w2v[:sample_size]  # можно сменить на X_ft или X_tfidf

# -----------------------------
# 2️⃣ Функция для оценки и вывода метрик
# -----------------------------
def evaluate_clusters(X, labels, name):
    if len(set(labels)) <= 1:  # Если все точки в одном кластере
        print(f"{name}: только один кластер")
        return
    sil = silhouette_score(X, labels)
    ch = calinski_harabasz_score(X, labels)
    db = davies_bouldin_score(X, labels)
    print(f"{name} → k={len(set(labels))}, Silhouette: {sil:.3f}, Calinski-Harabasz: {ch:.1f}, Davies-Bouldin: {db:.3f}")

# -----------------------------
# 3️⃣ Центроидные методы
# -----------------------------
k = 10  # пример
km = KMeans(n_clusters=k, random_state=42).fit(X_sample)
evaluate_clusters(X_sample, km.labels_, "KMeans")

mbkm = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=256).fit(X_sample)
evaluate_clusters(X_sample, mbkm.labels_, "MiniBatchKMeans")

# -----------------------------
# 4️⃣ Иерархические
# -----------------------------
agg = AgglomerativeClustering(n_clusters=k, linkage='ward').fit(X_sample)
evaluate_clusters(X_sample, agg.labels_, "Agglomerative Ward")

agg_avg = AgglomerativeClustering(n_clusters=k, linkage='average', metric='cosine').fit(X_sample)
evaluate_clusters(X_sample, agg_avg.labels_, "Agglomerative Average")

# -----------------------------
# 5️⃣ Плотностные
# -----------------------------
dbscan = DBSCAN(eps=0.5, min_samples=5).fit(X_sample)
evaluate_clusters(X_sample, dbscan.labels_, "DBSCAN")

hdb = hdbscan.HDBSCAN(min_cluster_size=10).fit(X_sample)
evaluate_clusters(X_sample, hdb.labels_, "HDBSCAN")

# -----------------------------
# 6️⃣ Вероятностные
# -----------------------------
gmm = GaussianMixture(n_components=k, random_state=42).fit(X_sample)
gmm_labels = gmm.predict(X_sample)
evaluate_clusters(X_sample, gmm_labels, "GMM")

# -----------------------------
# 7️⃣ Графовые
# -----------------------------
sc = SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=42).fit(X_sample)
evaluate_clusters(X_sample, sc.labels_, "SpectralClustering")

# -----------------------------
# 8️⃣ Визуализация UMAP
# -----------------------------
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
X_2d = reducer.fit_transform(X_sample)

plt.figure(figsize=(10,6))
plt.scatter(X_2d[:,0], X_2d[:,1], c=km.labels_, cmap='tab10', s=5)
plt.title("KMeans clusters visualized with UMAP")
plt.show()


In [ ]:
%%writefile clustering2.py
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.sparse import issparse
from tqdm import tqdm

def evaluate_clusters(X, labels, name, sample_size=500):
    n_clusters = len(set(labels))
    if n_clusters < 2:
        print(f"{name} → слишком мало кластеров для метрик")
        return
    # Если X разреженная, берем подвыборку для метрик
    if issparse(X):
        X_sample = X[:sample_size].toarray()
        labels_sample = labels[:sample_size]
    else:
        X_sample = X
        labels_sample = labels
    sil = silhouette_score(X_sample, labels_sample)
    ch = calinski_harabasz_score(X_sample, labels_sample)
    db = davies_bouldin_score(X_sample, labels_sample)
    print(f"{name} → k={n_clusters}, Silhouette: {sil:.3f}, Calinski-Harabasz: {ch:.1f}, Davies-Bouldin: {db:.3f}")

def run_all_clusters(X, k=10):
    # ------------------ Sparse-friendly ------------------
    if issparse(X):
        print("Sparse matrix detected → запускаем только KMeans / MiniBatchKMeans")
        kmeans = KMeans(n_clusters=k, random_state=42).fit(X)
        evaluate_clusters(X, kmeans.labels_, "KMeans")

        mbk = MiniBatchKMeans(n_clusters=k, random_state=42).fit(X)
        evaluate_clusters(X, mbk.labels_, "MiniBatchKMeans")
        return

    # ------------------ Dense ------------------
    print("Dense matrix detected → запускаем все алгоритмы")

    # Убираем нулевые векторы (важно для GloVe)
    nonzero_mask = ~(X == 0).all(axis=1)
    X_dense = X[nonzero_mask]

    # Если после фильтрации нет векторов, просто выходим
    if X_dense.shape[0] == 0:
        print("Все векторы нулевые → пропускаем кластеризацию")
        return

    # KMeans
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X_dense)
    evaluate_clusters(X_dense, kmeans.labels_, "KMeans")

    # MiniBatchKMeans
    mbk = MiniBatchKMeans(n_clusters=k, random_state=42).fit(X_dense)
    evaluate_clusters(X_dense, mbk.labels_, "MiniBatchKMeans")

    # Agglomerative
    agg_ward = AgglomerativeClustering(n_clusters=k, linkage='ward').fit(X_dense)
    evaluate_clusters(X_dense, agg_ward.labels_, "Agglomerative Ward")

    agg_avg = AgglomerativeClustering(n_clusters=k, linkage='average', metric='cosine').fit(X_dense)
    evaluate_clusters(X_dense, agg_avg.labels_, "Agglomerative Average")

    # DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=5, metric='cosine').fit(X_dense)
    evaluate_clusters(X_dense, dbscan.labels_, "DBSCAN")

    # HDBSCAN
    hdb = HDBSCAN(min_cluster_size=10, metric='euclidean').fit(X_dense)
    evaluate_clusters(X_dense, hdb.labels_, "HDBSCAN")

    # Gaussian Mixture
    gmm = GaussianMixture(n_components=k, random_state=42).fit(X_dense)
    labels_gmm = gmm.predict(X_dense)
    evaluate_clusters(X_dense, labels_gmm, "GMM")

    # Spectral Clustering
    spectral = SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=42).fit(X_dense)
    evaluate_clusters(X_dense, spectral.labels_, "SpectralClustering")


In [ ]:
from clustering2 import run_all_clusters

# Для TF-IDF (sparse)
run_all_clusters(X_tfidf, k=10)

# Для Word2Vec / FastText / GloVe (dense)
run_all_clusters(X_w2v, k=10)
run_all_clusters(X_ft, k=10)
run_all_clusters(X_glove, k=10)


In [ ]:
from clustering2 import run_all_clusters
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Пример: KMeans на плотной матрице
from sklearn.cluster import KMeans

k = 10
kmeans = KMeans(n_clusters=k, random_state=42).fit(X_w2v)
labels = kmeans.labels_

# Сохраняем в DataFrame
df_clusters = pd.DataFrame({'doc_id': range(len(labels)), 'cluster': labels})
print(df_clusters['cluster'].value_counts())

# Визуализируем через PCA
pca = PCA(n_components=2)
X_vis = pca.fit_transform(X_w2v)
plt.figure(figsize=(8,6))
plt.scatter(X_vis[:,0], X_vis[:,1], c=labels, cmap='tab10', s=5)
plt.title("KMeans на Word2Vec")
plt.show()



### **3.4. Оценка качества**
- **Внутренние метрики**: Silhouette, Calinski-Harabasz, Davies-Bouldin  
- **Внешние метрики** (если есть тематическая разметка из ЛР №3): ARI, NMI, V-measure  
- Постройте графики зависимости метрик от параметров


для графиков внутренних метрик

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.sparse import issparse

def plot_internal_metrics(X, k_range, title=""):
    sil_scores, ch_scores, db_scores = [], [], []

    # При разреженной матрице берём подвыборку (метрики требуют dense)
    if issparse(X):
        X_eval = X[:1500].toarray()
    else:
        X_eval = X

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42).fit(X_eval)
        labels = kmeans.labels_
        sil_scores.append(silhouette_score(X_eval, labels))
        ch_scores.append(calinski_harabasz_score(X_eval, labels))
        db_scores.append(davies_bouldin_score(X_eval, labels))

    plt.figure(figsize=(15,4))

    plt.subplot(1,3,1)
    plt.plot(k_range, sil_scores, marker='o')
    plt.title(f"Silhouette ({title})")

    plt.subplot(1,3,2)
    plt.plot(k_range, ch_scores, marker='o')
    plt.title(f"Calinski–Harabasz ({title})")

    plt.subplot(1,3,3)
    plt.plot(k_range, db_scores, marker='o')
    plt.title(f"Davies–Bouldin ({title})")

    plt.show()


📌 TF-IDF (разреженная)

In [ ]:
plot_internal_metrics(X_tfidf, k_range=range(2, 20, 2), title="TF-IDF")


📌 Word2Vec

In [ ]:
plot_internal_metrics(X_w2v, k_range=range(2, 20, 2), title="Word2Vec")


📌 FastText

In [ ]:
plot_internal_metrics(X_ft, k_range=range(2, 20, 2), title="FastText")


📌 GloVe

In [ ]:
plot_internal_metrics(X_glove, k_range=range(2, 20, 2), title="GloVe")


In [ ]:
# ------------------------------
# 1️⃣ Установки (если нужно)
# ------------------------------
!pip install gensim matplotlib --quiet

# ------------------------------
# 2️⃣ Импорты
# ------------------------------
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, v_measure_score
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

# ------------------------------
# 3️⃣ Генерация псевдо-меток LDA
# ------------------------------
# tokenized_docs = [doc.split() for doc in preprocessed_texts]  # должен быть у тебя
num_topics = 10  # количество «тем» для псевдо-разметки
dictionary = Dictionary(tokenized_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,
               passes=5, random_state=42)

# Получаем доминирующую тему для каждого документа
lda_labels = []
for bow in corpus:
    if len(bow) == 0:
        lda_labels.append(-1)  # пустой документ
    else:
        probs = lda.get_document_topics(bow)
        top = max(probs, key=lambda x: x[1])
        lda_labels.append(top[0])
lda_labels = np.array(lda_labels)
# Исключаем пустые документы
mask = lda_labels != -1
y_true = lda_labels[mask]

# ------------------------------
# 4️⃣ Функция расчёта внешних метрик и построения графиков
# ------------------------------
def plot_external_metrics(X, y_true, k_range, title=""):
    ari_scores, nmi_scores, v_scores = [], [], []

    # Если матрица разреженная, возьмём подвыборку для скорости
    from scipy.sparse import issparse
    if issparse(X):
        X_eval = X[:1500].toarray()
        y_eval = y_true[:1500]
    else:
        X_eval = X
        y_eval = y_true

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42).fit(X_eval)
        labels_pred = kmeans.labels_

        ari_scores.append(adjusted_rand_score(y_eval, labels_pred))
        nmi_scores.append(normalized_mutual_info_score(y_eval, labels_pred))
        v_scores.append(v_measure_score(y_eval, labels_pred))

    # Построим графики
    plt.figure(figsize=(15,4))

    plt.subplot(1,3,1)
    plt.plot(k_range, ari_scores, marker='o')
    plt.title(f"Adjusted Rand Index ({title})")

    plt.subplot(1,3,2)
    plt.plot(k_range, nmi_scores, marker='o')
    plt.title(f"Normalized Mutual Info ({title})")

    plt.subplot(1,3,3)
    plt.plot(k_range, v_scores, marker='o')
    plt.title(f"V-Measure ({title})")

    plt.show()

# ------------------------------
# 5️⃣ Запуск для твоих данных
# ------------------------------
k_range = range(2, 16)

# Пример: Word2Vec матрица
plot_external_metrics(X_w2v, y_true, k_range, title="Word2Vec")

# Можно повторить для X_tfidf, X_ft, X_glove
plot_external_metrics(X_tfidf, y_true, k_range, title="TF-IDF")
plot_external_metrics(X_ft, y_true, k_range, title="FastText")
plot_external_metrics(X_glove, y_true, k_range, title="GloVe")


### **3.5. Интерпретация**
- Для TF-IDF: топ-10 слов на кластер  
- Для эмбеддингов: ближайшие слова к центроиду (через `model.most_similar()`)  
- Визуализация: UMAP → 2D scatter plot с цветами кластеров

> 📌 **LIME и SHAP** могут применяться **только если кластеризация используется как признак в downstream-задаче** (например, кластер → признак для классификатора). В чистой кластеризации они **не применимы напрямую**, но могут использоваться для анализа косвенных эффектов.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, FastText
import umap

# -----------------------------
# 1️⃣ TF-IDF: топ-10 слов на кластер
# -----------------------------
def tfidf_top_words(X_tfidf, vectorizer, k=10):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X_tfidf)
    labels = kmeans.labels_
    feature_names = np.array(vectorizer.get_feature_names_out())

    print("TF-IDF: топ-10 слов на кластер")
    for cluster in range(k):
        # средний вес слов в кластере
        cluster_center = kmeans.cluster_centers_[cluster]
        top_idx = cluster_center.argsort()[-10:][::-1]
        top_words = feature_names[top_idx]
        print(f"Cluster {cluster}: {', '.join(top_words)}")
    return labels

# -----------------------------
# 2️⃣ Эмбеддинги: ближайшие слова к центроиду
# -----------------------------
def embeddings_top_words(model, X_emb, k=10, topn=10):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X_emb)
    labels = kmeans.labels_

    print("\nЭмбеддинги: ближайшие слова к центроиду")
    for cluster in range(k):
        centroid = kmeans.cluster_centers_[cluster]
        # most_similar может принимать вектор через положительный список
        try:
            similar = model.wv.similar_by_vector(centroid, topn=topn)
            words = [w for w,_ in similar]
            print(f"Cluster {cluster}: {', '.join(words)}")
        except:
            print(f"Cluster {cluster}: не удалось найти слова")
    return labels

# -----------------------------
# 3️⃣ UMAP: визуализация 2D scatter
# -----------------------------
def plot_umap(X, labels, title="UMAP 2D Visualization"):
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    X_2d = reducer.fit_transform(X)

    plt.figure(figsize=(10,8))
    scatter = plt.scatter(X_2d[:,0], X_2d[:,1], c=labels, cmap='tab10', s=10)
    plt.legend(*scatter.legend_elements(), title="Clusters")
    plt.title(title)
    plt.show()

# -----------------------------
# Пример использования
# -----------------------------
# TF-IDF
tfidf_labels = tfidf_top_words(X_tfidf, tfidf_vectorizer, k=10)
plot_umap(X_tfidf.toarray(), tfidf_labels, title="TF-IDF UMAP")

# Word2Vec
w2v_labels = embeddings_top_words(w2v_model, X_w2v, k=10)
plot_umap(X_w2v, w2v_labels, title="Word2Vec UMAP")

# FastText
ft_labels = embeddings_top_words(ft_model, X_ft, k=10)
plot_umap(X_ft, ft_labels, title="FastText UMAP")

# GloVe
glove_labels = embeddings_top_words(glove_model, X_glove, k=10)
plot_umap(X_glove, glove_labels, title="GloVe UMAP")



### **3.6. AutoML для кластеризации?**
> ❗ **Важно**: **AutoML-фреймворки (Auto-sklearn, TPOT, H2O)** **не поддерживают задачи без учителя** (включая кластеризацию) в стандартной конфигурации.  
> Auto-sklearn поддерживает **только задачи с разметкой** (классификация, регрессия).  
> Следовательно, **AutoML в данной работе НЕ ИСПОЛЬЗУЕТСЯ**.

AutoML не использовался, так как для кластеризации подходит только unsupervised подход, а AutoML рассчитан на supervised задачи.

### **3.7. Веб-интерфейс**
- Используйте `Streamlit` или `Gradio` для:
  - выбора метода токенизации/векторизации/алгоритма,  
  - визуализации кластеров,  
  - просмотра содержимого кластеров.


In [ ]:
%%writefile app4.py
import streamlit as st
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Получаем данные из session_state
X_options = {
    "TF-IDF": st.session_state.get('X_tfidf'),
    "Word2Vec": st.session_state.get('X_w2v'),
    "FastText": st.session_state.get('X_ft'),
    "GloVe": st.session_state.get('X_glove')
}
tokenized_docs = st.session_state.get('tokenized_docs', [])

st.title("Кластеризация текстов")

# Выбор метода векторизации
X_name = st.selectbox("Выберите метод векторизации", list(X_options.keys()))
n_clusters = st.slider("Количество кластеров", 2, 20, 10)

if st.button("Выполнить кластеризацию"):
    X = X_options[X_name]
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    labels = kmeans.labels_

    st.write(f"Результаты кластеризации для {X_name}:")

    # Просмотр примеров документов по кластерам
    cluster_id = st.number_input("Выберите кластер для просмотра примеров", 0, n_clusters-1, 0)
    examples = [doc for doc, label in zip(tokenized_docs, labels) if label == cluster_id][:10]
    for ex in examples:
        st.write(" ".join(ex))

    # Визуализация через PCA вместо UMAP
    pca = PCA(n_components=2, random_state=42)
    X_2d = pca.fit_transform(X)
    plt.figure(figsize=(6,4))
    plt.scatter(X_2d[:,0], X_2d[:,1], c=labels, cmap='tab10', s=10)
    plt.title(f"PCA scatter plot ({X_name})")
    st.pyplot(plt)


In [ ]:
# Очистка session_state
for key in list(st.session_state.keys()):
    del st.session_state[key]


In [ ]:
import streamlit as st

st.session_state['X_tfidf'] = X_tfidf[:1500].toarray() if hasattr(X_tfidf, "toarray") else X_tfidf[:1500]
st.session_state['X_w2v'] = X_w2v[:1500]
st.session_state['X_ft'] = X_ft[:1500]
st.session_state['X_glove'] = X_glove[:1500]
st.session_state['tokenized_docs'] = tokenized_docs[:1500]




In [ ]:
!streamlit run app4.py &>/dev/null &

from pyngrok import ngrok, conf
import time

NGROK_AUTH_TOKEN = "33rrvngq55Ym4ttNvRgvHJi6I9z_3uyg5NWWBepyuEtYCYiey"
conf.get_default().auth_token = NGROK_AUTH_TOKEN
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(addr="8501", proto="http")
print("Public URL для Streamlit:", public_url)

time.sleep(2)
print("Если страница пустая, проверь лог: !tail -n 200 /content/nohup.out")


In [ ]:
ngrok.kill()